# Setting up the environment (note: please use both R and python kernal for this script)

## Loading python packages

In [1]:
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import openpyxl
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from PIL import Image as PilImage
from openpyxl_image_loader import SheetImageLoader
import PIL 
import pandas as pd # load and manipulate data for One-Hot ncoding
import numpy as np # calculate the mean and std. dev. 
import matplotlib.pyplot as plt # to plot figures
import seaborn as sns

import os
os.environ['R_HOME'] = r'C:\Program Files\R\R-4.3.2'
r_home = os.environ.get("R_HOME")

!pip install rpy2 --quiet
import rpy2
import rpy2.robjects as ro
import rpy2.rinterface as ri
from rpy2.robjects import r

# Extra RPY2 items

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.vectors import IntVector, FloatVector
import rpy2.ipython.html
import rpy2.robjects.lib.ggplot2 as gp
from rpy2.ipython.ggplot import image_png
import rpy2.rinterface as rinterface

#rinterface.initr()
pandas2ri.activate()

%load_ext rpy2.ipython


plt.style.use("fivethirtyeight")

C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(
C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\rpy2\robjects\lib\ggplot2.py:70: UserWarning: This was designed againt ggplot2 versions starting with 3.3. but you have 3.4.4
  warnings.warn(


## Loading R packages

In [2]:
%%R

suppressPackageStartupMessages(library(Microsoft365R))
suppressPackageStartupMessages(library(strucchange))
suppressPackageStartupMessages(library(tseries))
suppressPackageStartupMessages(library(forecast))
suppressPackageStartupMessages(library(seasonal))
suppressPackageStartupMessages(library(seasonalview))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(openxlsx))
suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(Microsoft365R))
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(writexl))
suppressPackageStartupMessages(library(openxlsx))
suppressPackageStartupMessages(library(knitr))

# Loading data 

## Minor clean up

In [3]:
df = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Outputs\pred_sead_n_plots.xlsx', sheet_name = 'df_final'))
df_pred = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Outputs\df_pred_final.xlsx', sheet_name = 'df_pred'))
pd.options.display.max_columns = 500
# pd.options.display.max_rows = 500
df = df[df['index_china_gdp_sead'].notna()]
df['sv'] = ''

In [4]:
df

,Unnamed: 0,nm_oil_price,nm_oil_price_sead,wti_spot_price_sea,wti_spot_price_sead,brent_spot_price_sea,brent_spot_price_sead,dubai_price_sea,index_china_gdp_sea,index_china_gdp_sead,index_china_gdp_sead_1,index_us_gdp_sea,index_us_gdp_sead,index_cboe_volt_sea,us_unemploy_sea,us_unemploy_sead,index_ip_mining_sea,index_ip_mining_sead,index_cpi_china_sea,index_cpi_china_sead,opec_sea,opec_sead,non_opec_sea,non_opec_sead,index_cpi_us_sea,index_energy_sea,index_metal_sea,index_precious_sea,del_supply_nonopec_sea,del_supply_nonopec_sead,del_cap_per_sea,del_world_gdp_per_sea,del_supply_saudi_sea,opec_spare_capacity_sea,oecd_inventory_sea,oecd_inventory_sead,del_world_consump_sea,del_world_consump_sead,sv
2,2001-10-01,18.382252,-6.757264,21.387950,-4.342383,20.340461,-3.968467,19.839120,98.201105,-0.322019,0.006341,80.235000,0.227692,25.182127,5.462300,0.598185,83.137242,-1.049224,-0.073920,-0.878374,27098.500271,-860.532052,40354.480376,209.852757,0.008106,29.522924,24.809339,24.809339,0.721429,0.084463,1.138198,1.138198,-1.118946,5.075132,3951.160985,56.341081,1.554994,0.886441,
3,2002-01-01,19.688715,1.306463,21.815041,0.427090,21.362576,1.022115,21.054340,98.153885,-0.047220,0.274799,80.502035,0.267035,24.369796,5.711201,0.248901,80.764341,-2.372902,-0.557756,-0.483836,26212.007318,-886.492953,40519.732408,165.252032,0.073489,29.581693,26.429118,26.429118,1.046169,0.324740,0.941955,0.941955,-1.218224,6.418517,4001.609358,50.448373,2.429500,0.874506,
4,2002-04-01,24.628390,4.939675,26.129852,4.314811,24.899421,3.536845,22.390446,98.411143,0.257258,0.304478,80.783828,0.281793,22.733795,5.829171,0.117970,80.528715,-0.235625,-1.169446,-0.611690,26075.612187,-136.395130,41034.690700,514.958291,0.173884,31.382044,28.561953,28.561953,1.183076,0.136906,1.512703,1.512703,-0.638675,6.401378,3987.456819,-14.152539,2.672586,0.243086,
5,2002-07-01,26.950806,2.322416,27.257505,1.127654,25.754893,0.855471,25.571612,98.709637,0.298494,0.041236,81.182178,0.398350,30.216726,5.762944,-0.066227,80.263409,-0.265306,-0.754351,0.415096,26353.955434,278.343246,40822.331906,-212.358794,0.255128,32.301880,26.088024,26.088024,0.794765,-0.388310,0.861509,0.861509,-0.180427,6.009794,3936.463654,-50.993165,1.588576,-1.084011,
6,2002-10-01,26.055377,-0.895429,29.148620,1.891115,27.799405,2.044512,26.010084,98.976308,0.266671,-0.031823,81.629173,0.446995,33.527721,5.830142,0.067198,79.407091,-0.856318,-0.590316,0.164035,27097.899099,743.943666,40927.066553,104.734647,0.280038,34.560932,27.841472,27.841472,0.816391,0.021625,0.025518,0.025518,0.427349,4.452041,3889.245329,-47.218326,2.033835,0.445259,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2022-04-01,109.638961,15.537848,106.562443,12.255555,110.915370,11.090205,14.845543,100.335134,-0.260450,0.044907,126.824129,2.649138,27.390427,3.584704,-0.226442,112.525117,2.937890,2.282104,1.154428,26083.329211,45.331088,41170.269843,731.897998,0.777170,27.161063,19.973196,19.973196,1.577476,0.819325,1.599162,1.599162,-0.661343,6.410696,3974.115989,-28.527286,2.678788,0.184185,
85,2022-07-01,93.402468,-16.236493,93.363772,-13.198671,101.536098,-9.379271,90.634137,100.040376,-0.294758,-0.034308,128.271530,1.447401,26.293107,3.544451,-0.040253,115.709430,3.184313,2.663837,0.381733,26663.968052,580.638841,40799.884235,-370.385608,0.272594,27.104929,3.624211,3.624211,0.748440,-0.829036,0.838009,0.838009,-0.176009,5.511792,3934.546153,-39.569836,1.569917,-1.108870,
86,2022-10-01,82.709246,-10.693222,84.889991,-8.473781,90.800793,-10.735306,86.095092,99.633730,-0.406646,-0.111888,129.589687,1.318156,23.570406,3.627211,0.082760,115.788731,0.079301,1.759851,-0.903986,26972.564379,308.596327,40885.081341,85.197106,0.199640,21.486896,5.892569,5.892569,0.756295,0.007855,0.081845,0.081845,0.459791,4.370983,3899.590678,-34.955475,1.983745,0.413827,
87,2023-01-01,74.988884,-7.720362,76.167380,-8.722612,80.980203,-9.82

## Dropping non-regressors

In [5]:
df.drop(['nm_oil_price_sead', 'wti_spot_price_sea', 'brent_spot_price_sea', 'index_china_gdp_sea', 'index_china_gdp_sead', 'index_us_gdp_sea',\
         'us_unemploy_sea', 'index_ip_mining_sea', 'index_cpi_china_sea', 'opec_sea', 'non_opec_sea', 'del_supply_nonopec_sea', 'oecd_inventory_sea',\
         'del_world_consump_sea'], 
        axis = 1, inplace = True) # we set axis = 1 to remove columns, axis = 0 to remove rows. And inplace means we are modifying our dataframe, 
                                    # we ae not making a copy`

## Renaming variables for the ease of use

In [6]:
df.rename(columns = {"Unnamed: 0": "Date", "dubai_price_sea" : "dubai_price_sead", "index_cboe_volt_sea"  : "index_cboe_volt_sead", "index_cpi_us_sea" : "index_cpi_us_sead",\
                    "index_energy_sea" : "index_energy_sead", "index_metal_sea" : "index_metal_sead", "index_precious_sea" : "index_precious_sead","del_cap_per_sea"\
                    : "del_cap_per_sead", "del_world_gdp_per_sea": "del_world_gdp_per_sead", "del_supply_saudi_sea": "del_supply_saudi_sead",\
                    "opec_spare_capacity_sea": "opec_spare_capacity_sead", "index_china_gdp_sead_1": "index_china_gdp_sead"}, inplace = True)

# Creating structural break variables

## 911 attacks: sv_911

In [7]:
pd.set_option('mode.chained_assignment', None)
sv_911 = df.loc[(df['Date'] >= '2001-01-01') & (df['Date'] <= '2002-12-30')] 
sv_911['sv'] = ('sv_911')
sv_911['Date'] = pd.to_datetime(sv_911['Date'])
sv_911 = sv_911.set_index('Date')

## Low spare capacity: sv_lpc

In [8]:
pd.set_option('mode.chained_assignment', None)
sv_lpc = df.loc[(df['Date'] >= '2005-04-01') & (df['Date'] <= '2006-10-01')]
sv_lpc['sv'] = ('sv_lpc')
sv_lpc['Date'] = pd.to_datetime(sv_lpc['Date'])
sv_lpc = sv_lpc.set_index('Date')

## Global financial collapse: sv_gfc

In [9]:
pd.set_option('mode.chained_assignment', None)
sv_gfc = df.loc[(df['Date'] >= '2007-01-01') & (df['Date'] <= '2008-12-30')]
sv_gfc['sv'] = ('sv_gfc')
sv_gfc['Date'] = pd.to_datetime(sv_gfc['Date'])
sv_gfc = sv_gfc.set_index('Date')

## OPEC production cut: sv_opec

In [10]:
pd.set_option('mode.chained_assignment', None)
sv_opec = df.loc[(df['Date'] >= '2015-01-01') & (df['Date'] <= '2016-12-30')]
sv_opec['sv'] = ('sv_opec')
sv_opec['Date'] = pd.to_datetime(sv_opec['Date'])
sv_opec = sv_opec.set_index('Date')

## Global pandemic: sv_covid19

In [11]:
pd.set_option('mode.chained_assignment', None)
sv_covid19 = df.loc[(df['Date'] >= '2019-06-01') & (df['Date'] <= '2020-12-30')]
sv_covid19['sv'] = ('sv_covid19')
sv_covid19['Date'] = pd.to_datetime(sv_covid19['Date'])
sv_covid19 = sv_covid19.set_index('Date')

## Russia-ukrain conflict: sv_con22

In [12]:
pd.set_option('mode.chained_assignment', None)
sv_con22 = df.loc[(df['Date'] >= '2022-01-01') & (df['Date'] <= '2022-12-30')]
sv_con22['sv'] = ('sv_con22')
sv_con22['Date'] = pd.to_datetime(sv_con22['Date'])
sv_con22 = sv_con22.set_index('Date')

# Adding the structual variables to the dataset

In [13]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')


In [14]:
df = df.drop(pd.date_range('2001-01-01', '2002-12-30'), errors='ignore')
df = df.drop(pd.date_range('2005-04-01', '2006-10-01'), errors='ignore')
df = df.drop(pd.date_range('2007-01-01', '2008-12-30'), errors='ignore')
df = df.drop(pd.date_range('2015-01-01', '2016-12-30'), errors='ignore')
df = df.drop(pd.date_range('2019-06-01', '2020-12-30'), errors='ignore')
df = df.drop(pd.date_range('2022-01-01', '2022-12-30'), errors='ignore')

In [15]:
df = pd.concat([sv_911, sv_lpc, sv_gfc, sv_opec, sv_covid19, sv_con22, df], ignore_index = False)
df.sort_index(inplace=True)

In [16]:
df

,nm_oil_price,wti_spot_price_sead,brent_spot_price_sead,dubai_price_sead,index_china_gdp_sead,index_us_gdp_sead,index_cboe_volt_sead,us_unemploy_sead,index_ip_mining_sead,index_cpi_china_sead,opec_sead,non_opec_sead,index_cpi_us_sead,index_energy_sead,index_metal_sead,index_precious_sead,del_supply_nonopec_sead,del_cap_per_sead,del_world_gdp_per_sead,del_supply_saudi_sead,opec_spare_capacity_sead,oecd_inventory_sead,del_world_consump_sead,sv
Date,,,,,,,,,,,,,,,,,,,,,,,,
2001-10-01,18.382252,-4.342383,-3.968467,19.839120,0.006341,0.227692,25.182127,0.598185,-1.049224,-0.878374,-860.532052,209.852757,0.008106,29.522924,24.809339,24.809339,0.084463,1.138198,1.138198,-1.118946,5.075132,56.341081,0.886441,sv_911
2002-01-01,19.688715,0.427090,1.022115,21.054340,0.274799,0.267035,24.369796,0.248901,-2.372902,-0.483836,-886.492953,165.252032,0.073489,29.581693,26.429118,26.429118,0.324740,0.941955,0.941955,-1.218224,6.418517,50.448373,0.874506,sv_911
2002-04-01,24.628390,4.314811,3.536845,22.390446,0.304478,0.281793,22.733795,0.117970,-0.235625,-0.611690,-136.395130,514.958291,0.173884,31.382044,28.561953,28.561953,0.136906,1.512703,1.512703,-0.638675,6.401378,-14.152539,0.243086,sv_911
2002-07-01,26.950806,1.127654,0.855471,25.571612,0.041236,0.398350,30.216726,-0.066227,-0.265306,0.415096,278.343246,-212.358794,0.255128,32.301880,26.088024,26.088024,-0.388310,0.861509,0.861509,-0.180427,6.009794,-50.993165,-1.084011,sv_911
2002-10-01,26.055377,1.891115,2.044512,26.010084,-0.031823,0.446995,33.527721,0.067198,-0.856318,0.164035,743.943666,104.734647,0.280038,34.560932,27.841472,27.841472,0.021625,0.025518,0.025518,0.427349,4.452041,-47.218326,0.445259,sv_911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01,109.638961,12.255555,11.090205,14.845543,0.044907,2.649138,27.390427,-0.226442,2.937890,1.154428,45.331088,731.897998,0.777170,27.161063,19.973196,19.973196,0.819325,1.599162,1.599162,-0.661343,6.410696,-28.527286,0.184185,sv_con22
2022-07-01,93.402468,-13.198671,-9.379271,90.634137,-0.034308,1.447401,26.293107,-0.040253,3.184313,0.381733,580.638841,-370.385608,0.272594,27.104929,3.624211,3.624211,-0.829036,0.838009,0.838009,-0.176009,5.511792,-39.569836,-1.108870,sv_con22
2022-10-01,82.709246,-8.473781,-10.735306,86.095092,-0.111888,1.318156,23.570406,0.082760,0.079301,-0.903986,308.596327,85.197106,0.199640,21.486896,5.892569,5.892569,0.007855,0.081845,0.081845,0.459791,4.370983,-34.955475,0.413827,sv_con22


## Some last moment clean-up

In [17]:
df = pd.concat([df, pd.get_dummies(df['sv']).astype(int)], axis=1)
df.rename(columns = {"": "remove"}, inplace = True)
df.drop(['sv', 'remove'], axis = 1, inplace = True)

In [18]:
pd.options.display.max_rows = 500
df_pred

,Date,wti_spot_price_sead,brent_spot_price_sead,dubai_price_sead,index_china_gdp_sead,index_us_gdp_sead,index_cboe_volt_sead,us_unemploy_sead,index_ip_mining_sead,index_cpi_china_sead,opec_sead,non_opec_sead,index_cpi_us_sead,index_energy_sead,index_metal_sead,index_precious_sead,del_supply_nonopec_sead,del_cap_per_sead,del_world_gdp_per_sead,del_supply_saudi_sead,opec_spare_capacity_sead,oecd_inventory_sead,del_world_consump_sead,nm_oil_price,sv_911,sv_lpc,sv_gfc,sv_opec,sv_covid19,sv_con22
0,2023-07-30,-5.840083,2.325382,82.876366,-0.911323,1.194047,20.085571,-0.171856,4.561210,-0.307783,-13.773642,706.923950,0.131866,7.751447,13.733174,13.733174,-0.073360,-2.115845,-2.115845,1.474484,2.227800,2.757692,3.388950,79.028328,0,0,0,0,0,0
1,2023-10-28,-13.529203,-17.535038,74.711807,-0.164480,0.781384,22.119646,0.032615,3.015453,0.054584,806.919678,2.971413,-0.057139,12.629313,-6.686675,-6.686675,0.283199,-1.947022,-1.947022,1.927348,1.501382,-35.975090,-5.467120,71.919868,0,0,0,0,0,0
2,2024-01-26,-10.047746,-14.993237,77.074310,-0.108963,0.863851,23.169397,-0.101225,1.910647,0.683945,1118.634033,97.571190,0.237116,11.270504,0.301127,0.301127,0.081477,-0.815468,-0.815468,1.617720,2.235847,-98.245819,-3.689138,65.214203,0,0,0,0,0,0
3,2024-04-25,-14.548312,-9.541140,66.842728,9.595876,0.207882,25.247162,8.853168,-10.665100,-1.501626,-156.131210,-1795.554688,-0.141300,-5.010906,6.163105,6.163105,-3.530746,-2.195370,-2.195370,0.670179,2.168924,75.657715,0.222302,60.880001,0,0,0,0,0,0
4,2024-07-24,-0.026001,2.446545,71.064011,-5.025676,1.369176,24.254627,-2.321316,5.011416,-0.267677,-1409.199463,347.516571,0.037963,0.284925,15.108231,15.108231,-0.368389,-2.092887,-2.092887,1.448796,2.186764,-3.401475,8.998088,69.814995,0,0,0,0,0,0
5,2024-10-22,-11.831129,-8.741916,53.728775,-3.997951,0.753582,26.760496,-1.911838,5.352549,-1.986062,-538.038086,306.126404,-0.117739,3.307025,14.533134,14.533134,-0.831194,-1.981539,-1.981539,1.203316,1.805503,-57.237263,1.704587,66.358124,0,0,0,0,0,0
6,2025-01-20,-1.943241,-6.322901,85.684402,-0.932396,1.371162,20.213621,0.311554,3.368724,1.736481,-705.003235,1470.664062,0.402757,-0.649016,20.159712,20.159712,0.379608,-0.484047,-0.484047,0.641406,1.989799,-118.211975,-8.814223,81.763268,0,0,0,0,0,0
7,2025-04-20,-10.738132,-19.403624,76.611374,9.745985,0.157480,23.231710,8.768776,-7.022299,-1.773567,-244.874817,-1322.484253,-0.256893,4.207248,21.045147,21.045147,-3.111056,-2.063004,-2.063004,1.087119,2.161209,89.155891,1.801285,68.288437,0,0,0,0,0,0
8,2025-07-19,-5.628554,-0.521541,78.686539,-0.936194,1.157013,20.026726,-0.156212,2.049622,-0.684354,405.956390,425.725311,0.063843,2.887208,16.740032,16.740032,-0.290268,-2.388349,-2.388349,1.547062,2.199376,-6.100397,3.064528,76.439018,0,0,0,0,0,0
9,2025-10-17,-16.146349,-17.848160,72.447853,-0.076201,0.766359,22.851191,0.004479,5.822525,-1.971079,710.379089,-131.248123,-0.043890,4.780544,-6.594361,-6.594361,-0.244421,-1.853464,-1.853464,1.738008,1.814589,-33.823593,-6.059988,74.310738,0,0,0,0,0,0


# Exporting the dataset to an Excel file

In [19]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Outputs\df_final.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.

df.index = df.index.date
df.index.name = 'Date'
df.to_excel(writer, sheet_name='df')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['df']
writer.close()

# Merging the dataframes 

In [20]:
%%R

# odb <- get_business_onedrive()
# data_unadjusted <- odb$load_dataframe("Documents/Project Oil Price/dat.csv")
df_final <- read_xlsx("C:/Users/asifr/OneDrive - State of New Mexico/Documents/Project Oil Price/Outputs/df_final.xlsx")
df_pred <- read_xlsx("C:/Users/asifr/OneDrive - State of New Mexico/Documents/Project Oil Price/Outputs/df_pred_final.xlsx")

df_final<- rbind(df_final, df_pred)

# # Using openxlsx
suppressPackageStartupMessages(library(openxlsx))

OUT <- createWorkbook()

# ## adding some worksheet to the workbook OUT

addWorksheet(OUT, "df")

# ## Writing the data to the worksheets

writeData(OUT, sheet = "df", x = df_final)

                      
# # Exporting the file

saveWorkbook(OUT, "C:/Users/asifr/OneDrive - State of New Mexico/Documents/Project Oil Price/Outputs/df_final.xlsx", overwrite = TRUE)